# Storm Research Assistant

Reference
https://github.com/langchain-ai/langgraph/blob/main/examples/storm/storm.ipynb


In [1]:
# ## Prereqs

# %pip install -U langchain_community langchain_openai langgraph wikipedia  scikit-learn  langchain_fireworks
# # We use one or the other search engine below
# %pip install -U tavily-python
# %pip install -U duckduckgo-search
# # ! apt-get install graphviz graphviz-dev
# # %pip install pygraphviz



In [2]:
from storm import *
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


fast_llm = ChatOpenAI(model="gpt-3.5-turbo")
# long_context_llm = ChatOpenAI(model="gpt-4-turbo-preview")
long_context_llm = ChatOpenAI(model="gpt-3.5-turbo-0125")


embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore_dir = "./data/storm/vectorstore/"
vectorstore = Chroma(persist_directory=vectorstore_dir, embedding_function=embeddings)

interview_config = InterviewConfig(long_llm=long_context_llm, 
                                   fast_llm=fast_llm, 
                                   max_conversations=5, 
                                   max_reference_length=10000,
                                   tags_to_extract=[ "p", "h1", "h2", "h3"],
                                   embeddings=embeddings,
                                   vectorstore=vectorstore,
                                   vectorstore_dir=vectorstore_dir,
                                   runnable_config=RunnableConfig()
                                   )

In [3]:
# Needs topic as input - {"topic": ""}
outline = get_chain_outline(interview_config.fast_llm)

# Needs topic as input - {"topic": ""}
expand_chain = get_chain_expand_related_topics(fast_llm)


gen_perspectives_chain = get_chain_perspective_generator(fast_llm)

# Need messages as input - {"messages": []}
gen_queries_chain = get_chain_queries(fast_llm)
gen_answer_chain = get_chain_answer(fast_llm)

example_topic = "Increase you development productivity by using Docker compose and local docker labs"
t1 = {"topic": example_topic}

In [4]:
o1 = outline.invoke(t1)
o1

Outline(page_title='Increase Your Development Productivity with Docker Compose and Local Docker Labs', sections=[Section(section_title='Introduction to Docker Compose', description='Explanation of Docker Compose and its role in development environments.', subsections=None), Section(section_title='Benefits of Using Docker Compose', description='List of advantages that Docker Compose offers for development workflows.', subsections=None), Section(section_title='Setting Up Local Docker Labs', description='Step-by-step guide on how to create and manage local Docker labs for development purposes.', subsections=[Subsection(subsection_title='Installing Docker', description='Instructions on installing Docker on various operating systems.'), Subsection(subsection_title='Creating Docker Compose Files', description='Guidelines on creating Docker Compose files for efficient container management.'), Subsection(subsection_title='Running Docker Containers Locally', description='Demonstration on runnin

In [5]:
r1 = expand_chain.invoke(t1)
r1

RelatedSubjects(topics=['Docker (software)', 'Containerization', 'DevOps', 'Continuous Integration', 'Microservices'])

In [6]:
p1 = gen_perspectives_chain.invoke({"examples": r1.topics, "topic": example_topic})
p1

Perspectives(editors=[Editor(affiliation='Academic Institution', name='Dr. Container', role='Researcher', description='Dr. Container is an expert in containerization technologies and will focus on the historical development and technical aspects of Docker in the context of containerization.'), Editor(affiliation='Tech Company', name='DevOps Diana', role='DevOps Engineer', description='DevOps Diana will provide insights on the integration of Docker in DevOps practices, including automation, collaboration, and monitoring.'), Editor(affiliation='Open Source Community', name='CI Charlie', role='Continuous Integration Specialist', description='CI Charlie will emphasize the role of Docker in Continuous Integration pipelines, highlighting best practices and tools for seamless integration.'), Editor(affiliation='Software Development Firm', name='Microservices Mike', role='Microservices Architect', description='Microservices Mike will explore the use of Docker in building and deploying microser

In [7]:
c = get_chain_question_generator(fast_llm)
t2 = {"persona": ""}

q1 = c.invoke(t2)
q1

AIMessage(content='Can you provide some insights into the historical significance of the subject of my Wikipedia page?', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 138, 'total_tokens': 155}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-cfc7c4e3-2780-4e46-98c2-2a0eaa5be2a4-0')

In [8]:
q_in = {"messages": [HumanMessage(content=q1.content, name="JohnSmith")]}
a1 = await gen_queries_chain.ainvoke(q_in)
a1

Queries(queries=['historical significance of [subject] Wikipedia page', 'importance of [subject] in history', 'impact of [subject] on society'])

# Nodes

In [9]:
# Question node 
@as_runnable
async def node_generate_question(state: InterviewState) -> dict[str, Any]:
    """
    Generates a question for the editor in the interview.

    Args:
        state (InterviewState): The interview state.

    Returns:
        InterviewState: The updated interview state with the generated question added as a message.
    """
    editor: Editor = state.editor
    interview_config = state.interview_config
    fast_llm = interview_config.fast_llm

    # Normalize name
    name = cleanup_name(editor.name)
    editor.name = name


    logger.info(f'Generating question for {name}')
    gn_chain = c = get_chain_question_generator(fast_llm)
    input = {"persona": editor.persona}
    
    ai_response = await gn_chain.ainvoke(input)
    
    # Convert AI response to HumanMessage to simulate human conversation
    tag_with_name(ai_response, name)
    message = HumanMessage(**ai_response.dict(exclude={"type"}))
    
    state.messages.append(message)

    logger.info(f'Generated question for {name}: {message.content}')
    return state.as_dict()


# Test 

state = InterviewState(
    interview_config=interview_config,
    editor=Editor(affiliation="Example University", name="John Doe", role="Lead Editor", description="Experienced in the field of biology."),
    messages=[],
    references={}
)

await node_generate_question.ainvoke(state)

2024-04-20 17:54:49,116 [MainThread  ] [INFO ]  Generating question for JohnDoe
2024-04-20 17:54:49,624 [MainThread  ] [INFO ]  Generated question for JohnDoe: What recent studies in the field of biology do you find particularly groundbreaking or promising?


{'interview_config': InterviewConfig(long_llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11241f0d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x112422350>, model_name='gpt-3.5-turbo-0125', openai_api_key=SecretStr('**********'), openai_proxy=''), fast_llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x112253590>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1120ded90>, openai_api_key=SecretStr('**********'), openai_proxy=''), max_conversations=5, max_reference_length=10000, tags_to_extract=['p', 'h1', 'h2', 'h3'], embeddings=OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x112423490>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x11243a610>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', e

In [10]:

# @as_runnable
# async def node_fetch_data(state: InterviewState) -> dict[str, Any]:
#     """
#     Generates search engine queries for the editor in the interview and fetches relevant data. Data is started in a vector store.

#     Args:
#         state (InterviewState): The interview state.

#     Returns:
#         InterviewState: The updated interview state with the generated question added as a message.
#     """
#     editor: Editor = state.editor
#     config = state.interview_config
#     fast_llm = config.fast_llm
    
#     # Chain definitions
#     gen_answer_chain = get_chain_answer(fast_llm)
#     queries_chain = get_chain_queries(fast_llm)
    
#     # Normalize name
#     name = cleanup_name(editor.name)
#     # editor.name = name

#     logger.info(f'START - Generate answers for [{name}]')
    
#     # Generate search engine queries
    
#     q_in = {"messages": state.messages}
#     queries:Queries = await queries_chain.ainvoke(q_in)
#     logger.info(f"Got {len(queries.queries)} search engine queries for [{name}] -\n\t {queries.queries}")


#     # Run search engine on all generated queries using tool
#     query_results = await search_engine.abatch(queries.queries, config.runnable_config, return_exceptions=True)
#     successful_results = [res for res in query_results if not isinstance(res, Exception)]
#     all_query_results = {res["url"]: res["content"] for results in successful_results for res in results}

#     # to references
#     references: dict[str, Reference] = dict({k: Reference(url=k, title=v) for k, v in all_query_results.items()})
#     state.references = {**state.references, **references} # update references
    
#     # Dump search engine results to string and truncate to max reference length
#     dumped_successful_results = json.dumps(all_query_results)
    
#     logger.info(f"Got {len(successful_results)} search engine results for [{name}] - \n\t {all_query_results}")
#     logger.info(f"Dumped {len(dumped_successful_results)} characters for [{name}] - \n\t {dumped_successful_results}")

#     # # Append Questions from Wikipedia and Answers from the search engine
#     ai_message_for_queries: AIMessage = get_ai_message(json.dumps(queries.as_dict()))    
#     tool_results_message = generate_human_message(dumped_successful_results)
    
#     logger.debug(f"QUERY_AI_MSG: {ai_message_for_queries} for [{name}]")
#     logger.debug(f"RESULTS_H_MSG: {tool_results_message} for [{name}]")
#     state.messages.append(ai_message_for_queries)
#     state.messages.append(tool_results_message)

#     # Download data
#     # await fetch_pages_from_refs(references=references)

#     logger.info(f'Generated question for {name}: {message.content}')
#     return state.as_dict()


# # Test 

# state = InterviewState(
#     interview_config=interview_config,
#     editor=Editor(affiliation="Example University", name="John Doe", role="Lead Editor", description="Experienced in the field of biology."),
#     messages=[],
#     references={},
#     summary=""
# )

# await node_generate_question.ainvoke(state)

In [11]:
# Answer node

@as_runnable
async def node_generate_answer(state: InterviewState) -> dict[str, Any]:
    """
    Generates an answer for the editor's question in the interview.
    
    Args:
        state (InterviewState): The interview state.
    
    Returns:
        InterviewState: The updated interview state with the generated answer added as a message.
    """
    
    editor: Editor = state.editor
    config = state.interview_config
    fast_llm = config.fast_llm
    
    # Chain definitions
    gen_answer_chain = get_chain_answer(fast_llm)
    queries_chain = get_chain_queries(fast_llm)
    
    # Normalize name
    name = cleanup_name(editor.name)
    # editor.name = name

    logger.info(f'START - Generate answers for [{name}]')
    
    # Generate search engine queries
    
    q_in = {"messages": state.messages}
    queries:Queries = await queries_chain.ainvoke(q_in)
    logger.info(f"Got {len(queries.queries)} search engine queries for [{name}] -\n\t {queries.queries}")


    # Run search engine on all generated queries using tool
    query_results = await search_engine.abatch(queries.queries, config.runnable_config, return_exceptions=True)
    successful_results = [res for res in query_results if not isinstance(res, Exception)]
    all_query_results = {res["url"]: res["content"] for results in successful_results for res in results}
    
    # Dump search engine results to string and truncate to max reference length
    dumped_successful_results = json.dumps(all_query_results)
    # dumped_successful_results = dumped_successful_results[:config.max_reference_length] \
    #     if config.max_reference_length is not None \
    #     and len(dumped_successful_results) > int(config.max_reference_length) \
    #     else dumped_successful_results
    #     # and config.max_reference_length > 0 \
    
    logger.info(f"Got {len(successful_results)} search engine results for [{name}] - \n\t {all_query_results}")
    logger.info(f"Dumped {len(dumped_successful_results)} characters for [{name}] - \n\t {dumped_successful_results}")

    # # Append Questions from Wikipedia and Answers from the search engine
    ai_message_for_queries: AIMessage = get_ai_message(json.dumps(queries.as_dict()))    
    tool_results_message = generate_human_message(dumped_successful_results)
    
    logger.debug(f"QUERY_AI_MSG: {ai_message_for_queries} for [{name}]")
    logger.debug(f"RESULTS_H_MSG: {tool_results_message} for [{name}]")
    state.messages.append(ai_message_for_queries)
    state.messages.append(tool_results_message)
    
    # Only update the shared state with the final answer to avoid polluting the dialogue history with intermediate messages
    try:
        generated: AnswerWithCitations = await gen_answer_chain.ainvoke(state.as_dict())
        logger.info(f"Genreted final answer {generated} for [{name}] - \n\t {generated.as_str}")

    except Exception as e:
        logger.error(f"Error generating answer for [{name}] - {e}")
        generated = AnswerWithCitations(answer="", cited_urls=[])
    
    cited_urls = set(generated.cited_urls)
    
    # Update references with cited references - Save the retrieved information to a the shared state for future reference
    cited_references = {k: v for k, v in all_query_results.items() if k in cited_urls}
    # state.references = update_references(state.references, cited_references)
    state.references = {**state.references, **cited_references}
    
    
    # # Add message to shared state
    formatted_message = AIMessage(name=name, content=generated.as_str)
    state.messages.append(formatted_message)
    
    logger.info(f'END - generate answer for [{name}]')    
    return state.as_dict()

# clone state 
# ax1 = await node_generate_answer.ainvoke(state)
# ax1

In [12]:

def route_messages(state_dict: dict):
    
    print(f'Routing messages: {state_dict}')
    
    state = InterviewState.from_dict(state_dict)

    editor = state.editor
    config = state.interview_config
    name = cleanup_name(editor.name)

    print(f'Routing messages for [{name}]')

    messages = state.messages
    num_responses = len(
        [m for m in messages if isinstance(m, AIMessage) and m.name == name]
    )

    if num_responses >= config.max_conversations:
        return END
    
    last_question = messages[-2]
    last_question_content = str(last_question.content if last_question.content else "")
    if last_question_content.endswith("Thank you so much for your help!"):
        return END
    
    print(f'Continue asking question for [{name}] as this is not the last end of the conversation')
    return "ask_question"

In [13]:
class StormInterviewGraph1:
    def __init__(self, interview_config: InterviewConfig):
        self.interview_config = interview_config
        self.interviews = Interviews(interview_config=interview_config, perspectives=None, conversations={})
        self.graph = self.build_graph()
        
    def build_graph(self):
        builder = StateGraph(InterviewState)

        builder.add_node("ask_question", node_generate_question)
        builder.add_node("answer_question",node_generate_answer)
        builder.add_conditional_edges("answer_question", route_messages)
        builder.add_edge("ask_question", "answer_question")

        builder.set_entry_point("ask_question")
        return builder.compile().with_config(run_name="Conduct Interviews")
    
    def initialize_conversations(self):
        if self.interviews is not None \
            and self.interviews.perspectives is not None \
            and len(self.interviews.perspectives.editors) > 0:
            for editor in self.interviews.perspectives.editors:
                convo = InterviewState(interview_config=self.interview_config, editor=editor, messages=[], references={})
                self.interviews.conversations[editor] = convo
    
    async def run_single_interview(self, state: InterviewState):
        return await self.graph.ainvoke(state)
    
    
    async def run(self):
        inital_states = list(self.interviews.conversations.values())
        return await self.graph.abatch(inital_states, return_exceptions=True)
    

    async def stream_and_return_results(self, state):
        async for step in self.graph.astream(state):
            name = next(iter(step))
            print(name)
            print(f"Processing step: {name}")
            print("-- ", str(step[name]["messages"])[:300])
            if END in step:
                final_step = step
                
        final_state = next(iter(final_step.values()))
        return final_state

In [14]:
@as_runnable
async def survey_subjects(topic: str)-> Perspectives:
    print(f"Survey Subjects for Topic: {topic}")
    related_subjects = await expand_chain.ainvoke({"topic": topic})
    retrieved_docs = await wikipedia_retriever.abatch(
        related_subjects.topics, return_exceptions=True
    )
    all_docs = []
    for docs in retrieved_docs:
        if isinstance(docs, BaseException):
            continue
        all_docs.extend(docs)
    print(f"Retrieved {len(all_docs)} docs for Topic: {topic}")
    
    formatted = format_docs(all_docs)
    return await gen_perspectives_chain.ainvoke({"examples": formatted, "topic": topic})

In [15]:

interview_graph = StormInterviewGraph1(interview_config=interview_config)

# Generate perspectives
perspectives = await survey_subjects.ainvoke(example_topic)

# Set perspectives
interview_graph.interviews.perspectives = perspectives
interview_graph.initialize_conversations()
logger.info(interview_graph.interviews.conversations.keys())



Survey Subjects for Topic: Increase you development productivity by using Docker compose and local docker labs
Retrieved 6 docs for Topic: Increase you development productivity by using Docker compose and local docker labs


2024-04-20 17:55:02,051 [MainThread  ] [INFO ]  dict_keys([Editor(affiliation='Software Development Company', name='Alice Smith', role='Software Developer', description='Alice will focus on how Docker is used in software development, including its impact on project management and coding practices.'), Editor(affiliation='IT Operations Firm', name='Bob Johnson', role='IT Operations Specialist', description='Bob will provide insights into the operational aspects of Docker, such as deployment, monitoring, and troubleshooting in diverse environments.'), Editor(affiliation='Academic Institution', name='Dr. Emily Davis', role='Researcher', description='Dr. Davis will contribute an academic perspective on the implications of Docker in software engineering, focusing on its integration with Agile practices.'), Editor(affiliation='Freelance Consultant', name='Charlie Brown', role='DevOps Expert', description='Charlie will highlight the role of Docker in the DevOps methodology, emphasizing shared 

In [16]:
# Run interview

final_step = None
# await interview_graph.stream_and_return_results(
#     {"editor": perspectives.editors[0], "messages": []}
# )

initial_state : InterviewState = InterviewState(interview_config=interview_config, editor=p1.editors[0], messages=[], references={})
# {
#     "editor": p1.editors[0],
#     "messages": [
#         AIMessage(
#             content=f"So you said you were writing an article on {example_topic}?",
#             name="SubjectMatterExpert",
#         )
#     ],
# }
async for step in interview_graph.graph.astream(initial_state.as_dict()):
    name = next(iter(step))
    print(name)
    print(f"Processing step: {name}")
    print("-- ", str(step[name]["messages"])[:300])
    if END in step:
        final_step = step
        
final_state = next(iter(final_step.values()))

2024-04-20 17:55:02,084 [MainThread  ] [INFO ]  Generating question for DrContainer


2024-04-20 17:55:02,543 [MainThread  ] [INFO ]  Generated question for DrContainer: What are the key technical innovations that Docker introduced to the field of containerization?
2024-04-20 17:55:02,568 [MainThread  ] [INFO ]  START - Generate answers for [DrContainer]


ask_question
Processing step: ask_question
--  [HumanMessage(content='What are the key technical innovations that Docker introduced to the field of containerization?', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 185, 'total_tokens': 201}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'fini


2024-04-20 17:55:03,193 [MainThread  ] [INFO ]  Got 1 search engine queries for [DrContainer] -
	 ['Key technical innovations introduced by Docker in containerization']


Searching DuckDuckGo for [Key technical innovations introduced by Docker in containerization]


...

2024-04-20 17:55:33,199 [MainThread  ] [INFO ]  Got 1 search engine results for [DrContainer] - 
	 {'https://medium.com/@marwanjaber/docker-in-a-nutshell-a-practical-guide-to-containerization-62d0fce9761d': "Building a Docker Image with Our Node.js App. To containerize the Node.js application mentioned earlier, you'll need a Dockerfile. Create a new file named Dockerfile in the root directory of ...", 'https://medium.com/@hello_17906/docker-revolutionizing-software-development-and-containerization-cfdc9e705332': 'In 2013, Solomon Hykes introduced the world to Docker, a groundbreaking containerization technology that has since become a cornerstone of modern software development. This article traces the ...', 'https://www.redswitches.com/blog/containerization-using-docker/': 'Mastering Containerization Docker in 2024: A Quick Guide. Vasav. October 26, 2023. - 2 mins read. Imagine a world where the process of sending goods worldwide was chaotic. Each consignment requires specific packagin

Got search engine results: 5 for [Key technical advancements introduced by Docker in containerization]


2024-04-20 17:55:36,276 [MainThread  ] [INFO ]  Genreted final answer answer='Docker introduced key technical advancements to the field of containerization, revolutionizing software development. In 2013, Solomon Hykes introduced Docker as a groundbreaking containerization technology, which has become a cornerstone of modern software development. Docker simplified the process of deploying applications consistently across different environments by introducing containerization, enabling developers to package applications and dependencies into standardized units called containers. One of the key technologies utilized by Docker is namespaces, which allows the creation of isolated workspaces known as containers. When a container is executed, Docker generates a distinct set of namespaces specific to that container, enhancing isolation and security.' cited_urls=['https://medium.com/@hello_17906/docker-revolutionizing-software-development-and-containerization-cfdc9e705332', 'https://dev.to/yogi

Routing messages: InterviewState(interview_config=InterviewConfig(long_llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11241f0d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x112422350>, model_name='gpt-3.5-turbo-0125', openai_api_key=SecretStr('**********'), openai_proxy=''), fast_llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x112253590>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1120ded90>, openai_api_key=SecretStr('**********'), openai_proxy=''), max_conversations=5, max_reference_length=10000, tags_to_extract=['p', 'h1', 'h2', 'h3'], embeddings=OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x112423490>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x11243a610>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_ap

AttributeError: 'NoneType' object has no attribute 'values'

In [ ]:
# final_state = next(iter(step.values()))


In [ ]:
state2 = InterviewState.from_dict(final_state)
state2

InterviewState.from_dict: data is an instance of dict
InterviewState.from_dict: error: 'summary'


Traceback (most recent call last):
  File "/Users/tapiwamaruni/Documents/projects/llm/storm-researcher/storm/models.py", line 169, in from_dict
    summary=data["summary"]
            ~~~~^^^^^^^^^^^
KeyError: 'summary'


In [ ]:
# state2.trim_messages(max_characters=1000)
# final_state

TypeError: 'dict_values' object is not subscriptable

In [ ]:

## Generate Initial Outline

from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

direct_gen_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a Wikipedia writer. Write an outline for a Wikipedia page about a user-provided topic. Be comprehensive and specific.",
        ),
        ("user", "{topic}\n{format_instructions}"),
    ]
)


class Subsection(BaseModel):
    subsection_title: str = Field(..., title="Title of the subsection")
    description: str = Field(..., title="Content of the subsection")

    @property
    def as_str(self) -> str:
        return f"### {self.subsection_title}\n\n{self.description}".strip()


class Section(BaseModel):
    section_title: str = Field(..., title="Title of the section")
    description: str = Field(..., title="Content of the section")
    subsections: Optional[List[Subsection]] = Field(
        default=None,
        title="Titles and descriptions for each subsection of the Wikipedia page.",
    )

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            f"### {subsection.subsection_title}\n\n{subsection.description}"
            for subsection in self.subsections or []
        )
        return f"## {self.section_title}\n\n{self.description}\n\n{subsections}".strip()


class Outline(BaseModel):
    page_title: str = Field(..., title="Title of the Wikipedia page")
    sections: List[Section] = Field(
        default_factory=list,
        title="Titles and descriptions for each section of the Wikipedia page.",
    )

    @property
    def as_str(self) -> str:
        sections = "\n\n".join(section.as_str for section in self.sections)
        return f"# {self.page_title}\n\n{sections}".strip()


outline_parser = PydanticOutputParser(pydantic_object=Outline)

generate_outline_direct = direct_gen_outline_prompt.partial(format_instructions=outline_parser.get_format_instructions()) | fast_llm | outline_parser


In [ ]:

example_topic = "Impact of million-plus token context window language models on RAG"

initial_outline = generate_outline_direct.invoke({"topic": example_topic})

print(initial_outline.as_str)

In [ ]:
## Expand Topics\



In [ ]:
gen_related_topics_prompt = ChatPromptTemplate.from_template(
    """I'm writing a Wikipedia page for a topic mentioned below. Please identify and recommend some Wikipedia pages on closely related subjects. I'm looking for examples that provide insights into interesting aspects commonly associated with this topic, or examples that help me understand the typical content and structure included in Wikipedia pages for similar topics.

Please list the as many subjects and urls as you can.

Topic of interest: {topic}
{format_instructions}
"""
)


class RelatedSubjects(BaseModel):
    topics: List[str] = Field(
        description="Comprehensive list of related subjects as background research.",
    )


related_topics_parser = PydanticOutputParser(pydantic_object=RelatedSubjects)

expand_chain = gen_related_topics_prompt.partial(format_instructions=related_topics_parser.get_format_instructions()) | fast_llm | related_topics_parser


In [ ]:
related_subjects = await expand_chain.ainvoke({"topic": example_topic})
related_subjects

## Generate Perspectives

From these related subjects, we can select representative Wikipedia editors as "subject matter experts" with distinct backgrounds and affiliations. These will help distribute the search process to encourage a more well-rounded final report.


In [ ]:
class Editor(BaseModel):
    affiliation: str = Field(
        description="Primary affiliation of the editor.",
    )
    name: str = Field(
        description="Name of the editor.",
    )
    role: str = Field(
        description="Role of the editor in the context of the topic.",
    )
    description: str = Field(
        description="Description of the editor's focus, concerns, and motives.",
    )

    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"


class Perspectives(BaseModel):
    editors: List[Editor] = Field(
        description="Comprehensive list of editors with their roles and affiliations.",
        # Add a pydantic validation/restriction to be at most M editors
    )

gen_perspectives_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You need to select a diverse (and distinct) group of Wikipedia editors who will work together to create a comprehensive article on the topic. Each of them represents a different perspective, role, or affiliation related to this topic.\
    You can use other Wikipedia pages of related topics for inspiration. For each editor, add a description of what they will focus on.

    Wiki page outlines of related topics for inspiration:
    {examples}""",
        ),
        ("user", "Topic of interest: {topic}\n\n{format_instructions}"),
    ]
)

perspectives_parser = PydanticOutputParser(pydantic_object=Perspectives)

gen_perspectives_chain = gen_perspectives_prompt.partial(format_instructions=perspectives_parser.get_format_instructions()) | fast_llm | perspectives_parser


In [ ]:
from langchain_community.retrievers import WikipediaRetriever
from langchain_core.runnables import RunnableLambda, chain as as_runnable

wikipedia_retriever = WikipediaRetriever(load_all_available_meta=True, top_k_results=1)


def format_doc(doc, max_length=1000)-> str:
    related = "- ".join(doc.metadata["categories"])
    return f"### {doc.metadata['title']}\n\nSummary: {doc.page_content}\n\nRelated\n{related}"[
        :max_length
    ]


def format_docs(docs):
    return "\n\n".join(format_doc(doc) for doc in docs)


@as_runnable
async def survey_subjects(topic: str)-> Perspectives:
    print(f"Survey Subjects for Topic: {topic}")
    related_subjects = await expand_chain.ainvoke({"topic": topic})
    retrieved_docs = await wikipedia_retriever.abatch(
        related_subjects.topics, return_exceptions=True
    )
    all_docs = []
    for docs in retrieved_docs:
        if isinstance(docs, BaseException):
            continue
        all_docs.extend(docs)
    print(f"Retrieved {len(all_docs)} docs for Topic: {topic}")
    
    formatted = format_docs(all_docs)
    return await gen_perspectives_chain.ainvoke({"examples": formatted, "topic": topic})

In [ ]:
perspectives = await survey_subjects.ainvoke(example_topic)


In [ ]:

perspectives.dict()


## Expert Dialog

Each wikipedia writer is primed to role-play using the perspectives presented above. It will ask a series of questions of a second "domain expert" with access to a search engine. This generate content to generate a refined outline as well as an updated index of reference documents.

### Interview State

The conversation is cyclic, so we will construct it within its own graph. The State will contain messages, the reference docs, and the editor (with its own "persona") to make it easy to parallelize these conversations.


In [ ]:
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated, Sequence


def add_messages(left, right):
    if not isinstance(left, list):
        left = [left]
    if not isinstance(right, list):
        right = [right]
    return left + right


def update_references(references, new_references):
    if not references:
        references = {}
    references.update(new_references)
    return references


def update_editor(editor, new_editor):
    # Can only set at the outset
    if not editor:
        return new_editor
    return editor


class InterviewState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    references: Annotated[Optional[dict], update_references]
    editor: Annotated[Optional[Editor], update_editor]

# Dialog Roles

The graph will have two participants: the wikipedia editor (generate_question), who asks questions based on its assigned role, and a domain expert (`gen_answer_chain), who uses a search engine to answer the questions as accurately as possible.


In [ ]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage


gen_qn_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an experienced Wikipedia writer and want to edit a specific page. \
Besides your identity as a Wikipedia writer, you have a specific focus when researching the topic. \
Now, you are chatting with an expert to get information. Ask good questions to get more useful information.

When you have no more questions to ask, say "Thank you so much for your help!" to end the conversation.\
Please only ask one question at a time and don't ask what you have asked before.\
Your questions should be related to the topic you want to write.
Be comprehensive and curious, gaining as much unique insight from the expert as possible.\

Stay true to your specific perspective:

{persona}""",
        ),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)


def tag_with_name(ai_message: AIMessage, name: str) -> AIMessage:
    ai_message.name = name
    return ai_message


def swap_roles(state: InterviewState, name: str) -> InterviewState:

    # Normalize name
    name = cleanup_name(name)

    print(f'Swapping roles for {name}')

    converted = []
    for message in state["messages"]:
        if isinstance(message, AIMessage) and message.name != name:
            message = HumanMessage(**message.dict(exclude={"type"}))
        converted.append(message)
    
    print(f'Converted messages for {name} while swapping roles: {len(converted)} messages')

    return {"messages": converted}


@as_runnable
async def generate_question(state: InterviewState) -> InterviewState:
    editor = state["editor"]

    name = cleanup_name(editor.name)

    print(f'Generating question for {name}')

    gn_chain = (
        RunnableLambda(swap_roles).bind(name=name)
        | gen_qn_prompt.partial(persona=editor.persona)
        | fast_llm
        | RunnableLambda(tag_with_name).bind(name=name)
    )
    result:AIMessage = await gn_chain.ainvoke(state)

    print(f'Generated question for {name}')
    return {"messages": [result]}

In [ ]:
messages = [
    HumanMessage(f"So you said you were writing an article on {example_topic}?")
]
question = await generate_question.ainvoke(
    {
        "editor": perspectives.editors[0],
        "messages": messages,
    }
)

question["messages"][0]

### Answer questions

The `gen_answer_chain` first generates queries (query expansion) to answer the editor's question, then responds with citations.


In [ ]:
class Queries(BaseModel):
    queries: List[str] = Field(
        description="Comprehensive list of search engine queries to answer the user's questions.",
    )


gen_queries_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful research assistant. Query the search engine to answer the user's questions.\n{format_instructions}",
        ),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)

queries_parser = PydanticOutputParser(pydantic_object=Queries)

gen_queries_chain = gen_queries_prompt.partial(format_instructions=queries_parser.get_format_instructions()) | fast_llm | queries_parser

In [ ]:

queries = await gen_queries_chain.ainvoke(
    {"messages": [HumanMessage(content=question["messages"][0].content)]}
)

queries

In [ ]:

class AnswerWithCitations(BaseModel):
    answer: str = Field(
        description="Comprehensive answer to the user's question with citations.",
    )
    cited_urls: List[str] = Field(
        description="List of urls cited in the answer.",
    )

    @property
    def as_str(self) -> str:
        return f"{self.answer}\n\nCitations:\n\n" + "\n".join(
            f"[{i+1}]: {url}" for i, url in enumerate(self.cited_urls)
        )


gen_answer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert who can use information effectively. You are chatting with a Wikipedia writer who wants\
 to write a Wikipedia page on the topic you know. You have gathered the related information and will now use the information to form a response.

Make your response as informative as possible and make sure every sentence is supported by the gathered information.
Each response must be backed up by a citation from a reliable source, formatted as a footnote, reproducing the URLS after your response.
{format_instructions}""",
        ),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)

ac_parser = PydanticOutputParser(pydantic_object=AnswerWithCitations)

gen_answer_chain = gen_answer_prompt.partial(format_instructions=ac_parser.get_format_instructions()) | fast_llm | ac_parser 

# .with_structured_output(
#     AnswerWithCitations, include_raw=True
# ).with_config(run_name="GenerateAnswer")
                                             

In [ ]:
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain_core.tools import tool

# DDG 
search_engine = DuckDuckGoSearchAPIWrapper()

@tool
async def search_engine(query: str):
    """Search engine to the internet."""

    print(f"Searching DuckDuckGo for [{query}]")

    results = DuckDuckGoSearchAPIWrapper()._ddgs_text(query)

    print(f"Got search engine results: {len(results)} for [{query}]")
    
    return [{"content": r["body"], "url": r["href"]} for r in results]

In [ ]:
from langchain_core.runnables import RunnableConfig
import json, re


async def gen_answer(
    state: InterviewState,
    config: Optional[RunnableConfig] = None,
    name: str = "SubjectMatterExpert",
    max_str_len: int = 15000,
):
    name = cleanup_name(name)

    print(f'Generating answers for [{name}]')


    swapped_state = swap_roles(state, name)  # Convert all other AI messages
    
    queries:Queries = await gen_queries_chain.ainvoke(swapped_state)

    print(f"Got {len(queries.queries)} search engine queries for [{name}]")

    query_results = await search_engine.abatch(
        queries.queries, config, return_exceptions=True
    )
    successful_results = [
        res for res in query_results if not isinstance(res, Exception)
    ]

    print(f"Got {len(successful_results)} search engine results for [{name}]")

    all_query_results = {
        res["url"]: res["content"] for results in successful_results for res in results
    }

    # We could be more precise about handling max token length if we wanted to here
    dumped = json.dumps(all_query_results)[:max_str_len]
    
    ai_message: AIMessage = str(queries)
    # print(f"Got {ai_message} for [{name}]")
    
    # tool_call = queries["raw"].additional_kwargs["tool_calls"][0]
    # tool_id = tool_call["id"]

    # tool_message = ToolMessage(tool_call_id=tool_id, content=dumped)
    tool_message = HumanMessage(content=dumped)

    swapped_state["messages"].extend([ai_message, tool_message])
    
    # Only update the shared state with the final answer to avoid
    # polluting the dialogue history with intermediate messages
    try:
        generated: AnswerWithCitations = await gen_answer_chain.ainvoke(swapped_state)
    except Exception as e:
        print(f"Error generating answer for [{name}] - {e}")
        generated = AnswerWithCitations(answer="", cited_urls=[])
    
    cited_urls = set(generated.cited_urls)
    
    # Save the retrieved information to a the shared state for future reference
    cited_references = {k: v for k, v in all_query_results.items() if k in cited_urls}
    
    formatted_message = AIMessage(name=name, content=generated.as_str)

    print(f'Finished generating answer for [{name}]')
    return {"messages": [formatted_message], "references": cited_references}
    

In [ ]:

example_answer = await gen_answer(
    {"messages": [HumanMessage(content=question["messages"][0].content)]}
)
example_answer["messages"][-1].content

# Construct the Interview Graph

Now that we've defined the editor and domain expert, we can compose them in a graph.


In [ ]:
max_num_turns = 5




builder = StateGraph(InterviewState)

builder.add_node("ask_question", generate_question)
builder.add_node("answer_question", gen_answer)
builder.add_conditional_edges("answer_question", route_messages)
builder.add_edge("ask_question", "answer_question")

builder.set_entry_point("ask_question")
interview_graph = builder.compile().with_config(run_name="Conduct Interviews")

In [ ]:
from IPython.display import Image

# comment out if you have not installed pygraphviz
# Image(interview_graph.get_graph().draw_png())

In [ ]:

final_step = None

initial_state = {
    "editor": perspectives.editors[0],
    "messages": [
        AIMessage(
            content=f"So you said you were writing an article on {example_topic}?",
            name="SubjectMatterExpert",
        )
    ],
}
async for step in interview_graph.astream(initial_state):
    name = next(iter(step))
    print(name)
    print(f"Processing step: {name}")
    print("-- ", str(step[name]["messages"])[:300])
    if END in step:
        final_step = step
        
final_state = next(iter(final_step.values()))

In [ ]:
final_state

## Refine Outline

At this point in STORM, we've conducted a large amount of research from different perspectives. It's time to refine the original outline based on these investigations. Below, create a chain using the LLM with a long context window to update the original outline.


In [ ]:
refine_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a Wikipedia writer. You have gathered information from experts and search engines. Now, you are refining the outline of the Wikipedia page. \
You need to make sure that the outline is comprehensive and specific. \
Topic you are writing about: {topic} 

Old outline:

{old_outline}
""",
        ),
        (
            "user",
            "Refine the outline based on your conversations with subject-matter experts:\n\nConversations:\n\n{conversations}\n\n{format_instructions}\n\nWrite the refined Wikipedia outline:",
        ),
    ]
)


# Using turbo preview since the context can get quite long
refine_outline_chain = refine_outline_prompt.partial(format_instructions=outline_parser.get_format_instructions()) | long_context_llm | outline_parser

In [ ]:
refined_outline = refine_outline_chain.invoke(
    {
        "topic": example_topic,
        "old_outline": initial_outline.as_str,
        "conversations": "\n\n".join(
            f"### {m.name}\n\n{m.content}" for m in final_state["messages"]
        ),
    }
)

In [ ]:
print(refined_outline.as_str)

In [ ]:
## Generate Article

In [ ]:
from langchain_core.documents import Document

from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
reference_docs = [
    Document(page_content=v, metadata={"source": k})
    for k, v in final_state["references"].items()
]

print(f"Number of references: {len(reference_docs)}")

# This really doesn't need to be a vectorstore for this size of data.
# It could just be a numpy matrix. Or you could store documents
# across requests if you want.
vectorstore = SKLearnVectorStore.from_documents(
    reference_docs,
    embedding=embeddings,
)
retriever = vectorstore.as_retriever(k=10)

In [ ]:
retriever.invoke("What's a long context LLM anyway?")

#### Generate Sections

Now you can generate the sections using the indexed docs.


In [ ]:
class SubSection(BaseModel):
    subsection_title: str = Field(..., title="Title of the subsection")
    content: str = Field(
        ...,
        title="Full content of the subsection. Include [#] citations to the cited sources where relevant.",
    )

    @property
    def as_str(self) -> str:
        return f"### {self.subsection_title}\n\n{self.content}".strip()


class WikiSection(BaseModel):
    section_title: str = Field(..., title="Title of the section")
    content: str = Field(..., title="Full content of the section")
    subsections: Optional[List[Subsection]] = Field(
        default=None,
        title="Titles and descriptions for each subsection of the Wikipedia page.",
    )
    citations: List[str] = Field(default_factory=list)

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            subsection.as_str for subsection in self.subsections or []
        )
        citations = "\n".join([f" [{i}] {cit}" for i, cit in enumerate(self.citations)])
        return (
            f"## {self.section_title}\n\n{self.content}\n\n{subsections}".strip()
            + f"\n\n{citations}".strip()
        )


section_writer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert Wikipedia writer. Complete your assigned WikiSection from the following outline:\n\n"
            "{outline}\n\nCite your sources, using the following references:\n\n<Documents>\n{docs}\n<Documents>",
        ),
        ("user", "Write the full WikiSection for the {section} section.\n{format_instructions}"),
    ]
)


async def retrieve(inputs: dict):
    docs = await retriever.ainvoke(inputs["topic"] + ": " + inputs["section"])
    formatted = "\n".join(
        [
            f'<Document href="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )
    return {"docs": formatted, **inputs}

wiki_parser = PydanticOutputParser(pydantic_object=WikiSection)

section_writer = (
    retrieve
    | section_writer_prompt.partial(format_instructions=wiki_parser.get_format_instructions())
    | long_context_llm
    | wiki_parser
)

In [ ]:
section = await section_writer.ainvoke(
    {
        "outline": refined_outline.as_str,
        "section": refined_outline.sections[1].section_title,
        "topic": example_topic,
    }
)
print(section.as_str)

#### Generate final article

Now we can rewrite the draft to appropriately group all the citations and maintain a consistent voice.


In [ ]:
from langchain_core.output_parsers import StrOutputParser

writer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert Wikipedia author. Write the complete wiki article on {topic} using the following section drafts:\n\n"
            "{draft}\n\nStrictly follow Wikipedia format guidelines.",
        ),
        (
            "user",
            'Write the complete Wiki article using markdown format. Organize citations using footnotes like "[1]","" avoiding duplicates in the footer. Include URLs in the footer.',
        ),
    ]
)

writer = writer_prompt | long_context_llm | StrOutputParser()

In [ ]:
for tok in writer.stream({"topic": example_topic, "draft": section.as_str}):
    print(tok, end="")

## Final Flow

Now it's time to string everything together. We will have 6 main stages in sequence:
.

1. Generate the initial outline + perspectives
2. Batch converse with each perspective to expand the content for the article
3. Refine the outline based on the conversations
4. Index the reference docs from the conversations
5. Write the individual sections of the article
6. Write the final wiki

The state tracks the outputs of each stage.


In [ ]:
class ResearchState(TypedDict):
    topic: str
    outline: Outline
    editors: List[Editor]
    interview_results: List[InterviewState]
    # The final sections output
    sections: List[WikiSection]
    article: str

In [ ]:
import asyncio


async def initialize_research(state: ResearchState):
    topic = state["topic"]
    coros = (
        generate_outline_direct.ainvoke({"topic": topic}),
        survey_subjects.ainvoke(topic),
    )
    results = await asyncio.gather(*coros)
    return {
        **state,
        "outline": results[0],
        "editors": results[1].editors,
    }


async def conduct_interviews(state: ResearchState):
    topic = state["topic"]
    initial_states = [
        {
            "editor": editor,
            "messages": [
                AIMessage(
                    content=f"So you said you were writing an article on {topic}?",
                    name="SubjectMatterExpert",
                )
            ],
        }
        for editor in state["editors"]
    ]
    # We call in to the sub-graph here to parallelize the interviews
    interview_results = await interview_graph.abatch(initial_states)

    return {
        **state,
        "interview_results": interview_results,
    }


def format_conversation(interview_state):
    messages = interview_state["messages"]
    convo = "\n".join(f"{m.name}: {m.content}" for m in messages)
    return f'Conversation with {interview_state["editor"].name}\n\n' + convo


async def refine_outline(state: ResearchState):
    convos = "\n\n".join(
        [
            format_conversation(interview_state)
            for interview_state in state["interview_results"]
        ]
    )

    updated_outline = await refine_outline_chain.ainvoke(
        {
            "topic": state["topic"],
            "old_outline": state["outline"].as_str,
            "conversations": convos,
        }
    )
    return {**state, "outline": updated_outline}


async def index_references(state: ResearchState):
    all_docs = []
    for interview_state in state["interview_results"]:
        reference_docs = [
            Document(page_content=v, metadata={"source": k})
            for k, v in interview_state["references"].items()
        ]
        all_docs.extend(reference_docs)
    await vectorstore.aadd_documents(all_docs)
    return state


async def write_sections(state: ResearchState):
    outline = state["outline"]
    sections = await section_writer.abatch(
        [
            {
                "outline": refined_outline.as_str,
                "section": section.section_title,
                "topic": state["topic"],
            }
            for section in outline.sections
        ]
    )
    return {
        **state,
        "sections": sections,
    }


async def write_article(state: ResearchState):
    topic = state["topic"]
    sections = state["sections"]
    draft = "\n\n".join([section.as_str for section in sections])
    article = await writer.ainvoke({"topic": topic, "draft": draft})
    return {
        **state,
        "article": article,
    }

#### Create the graph


In [ ]:
builder_of_storm = StateGraph(ResearchState)

nodes = [
    ("init_research", initialize_research),
    ("conduct_interviews", conduct_interviews),
    ("refine_outline", refine_outline),
    ("index_references", index_references),
    ("write_sections", write_sections),
    ("write_article", write_article),
]
for i in range(len(nodes)):
    name, node = nodes[i]
    builder_of_storm.add_node(name, node)
    if i > 0:
        builder_of_storm.add_edge(nodes[i - 1][0], name)

builder_of_storm.set_entry_point(nodes[0][0])
builder_of_storm.set_finish_point(nodes[-1][0])
storm = builder_of_storm.compile()

In [ ]:
async for step in storm.astream(
    {
        "topic": "Building better slack bots using LLMs",
    }
):
    name = next(iter(step))
    print(name)
    print("-- ", str(step[name])[:300])
    if END in step:
        results = step

In [ ]:
article = results[END]["article"]

## Render the Wiki

Now we can render the final wiki page!


In [ ]:
from IPython.display import Markdown

# We will down-header the sections to create less confusion in this notebook
Markdown(article.replace("\n#", "\n##"))